In [1]:
from temple.session import TUSession
from dotenv import load_dotenv
from os import getenv

load_dotenv(override=True)


session = TUSession()
session.login(username=getenv("USERNAME"), password=getenv("PASSWORD"))

In [37]:
from typing_extensions import Annotated
from pydantic import BaseModel, StringConstraints
from typing import Optional, Union, Tuple, Literal
from school.courses import CourseSection
from util.print import pprint

CourseConstraint = Annotated[str, StringConstraints(pattern=r"^[A-Z]+\d+$")]
SectionConstraint = Annotated[str, StringConstraints(pattern=r"^\d+$")]
TeacherConstraint = Annotated[str, StringConstraints(strip_whitespace=True, to_lower=True)]


class CourseSelect(BaseModel, frozen=True):
    course: CourseConstraint
    section: Optional[SectionConstraint] = None


class CourseIgnore(BaseModel, frozen=True):
    course: CourseConstraint
    section: Optional[Union[Tuple[SectionConstraint], SectionConstraint]] = None
    teacher: Optional[Union[Tuple[TeacherConstraint], TeacherConstraint]] = None
    instructional_method: Optional[Literal["CLAS", "OLL"]] = None
    waitlist: Optional[bool] = None

    def should_ignore(self, _course_section: CourseSection) -> bool:
        """
        Determine if the course section should be ignored based on section, teacher, or instructional method.
        """

        # Check section constraint
        if self.section:
            section_numbers = self.section if isinstance(self.section, list) else [self.section]
            if _course_section.sequenceNumber in section_numbers:
                return True

        # Check teacher constraint
        if self.teacher:
            faculty_names = {faculty.get_name().lower() for faculty in _course_section.faculty}
            teacher_names = set(self.teacher) if isinstance(self.teacher, list) else {self.teacher}
            if faculty_names & teacher_names:
                return True

        # Check instructional method constraint
        if self.instructional_method == _course_section.instructionalMethod:
            return True

        # Check waitlist method constraint
        if self.waitlist is not None:
            if self.waitlist:
                if _course_section.seatsAvailable == 0:
                    return True
            else:
                if _course_section.seatsAvailable > 0:
                    return True

        return False


In [ ]:
from school.session import SchoolSession
from school.courses import CourseSection
from functools import cache
from itertools import product
from typing import Dict, List, Set
from temple.session import TUSession


class CourseBuilder:
    _session: SchoolSession
    _term: int
    _courses_select: Set[CourseSelect]
    _courses_ignore: Dict[str, Set[CourseIgnore]]

    def __init__(self, _session: SchoolSession):
        self._session = _session
        self._courses_select = set()
        self._courses_ignore = {}
        self._term = -1

    def print_terms(self, _max: int):
        assert self._term > 0, "term not selected"
        terms = self._session.get_terms(_max)

        for term in terms:
            print(f"CODE: {term.code}, DESCRIPTION: {term.description}")

    def select_term(self, _term: int):
        self._term = _term

    def update(self, _course_options: List[Union[CourseSelect, CourseIgnore]]):
        for course_option in _course_options:
            if isinstance(course_option, CourseSelect):
                self._courses_select.add(course_option)
            elif isinstance(course_option, CourseIgnore):
                self._courses_ignore.setdefault(course_option.course, set()).add(course_option)

    def get_combinations(self) -> List[List[CourseSection]]:
        assert self._term > 0, "term not selected"

        def section_filter(_section: CourseSection):
            # Make sure only main campus classes classes are allowed
            if _section.campusDescription != "Main":
                return False

            # Make sure only in-person clasess and online classes are allowed
            if _section.instructionalMethod != "CLAS" and _section.instructionalMethod != "OLL":
                return False

            # Make sure all unwanted courses are ignored
            if _section.subjectCourse in self._courses_ignore:
                return not any(
                    course.should_ignore(_section) for course in self._courses_ignore[_section.subjectCourse]
                )

            # All checks passed
            return True

        def overlap_filter(_sections: List[CourseSection]):
            for i in range(len(_sections)):
                for j in range(i + 1, len(_sections)):
                    if _sections[i].overlaps(_sections[j]):
                        return False
            return True

        all_sections: List[List[CourseSection]] = []

        for selected_course in self._courses_select:
            try:
                course_sections = self._session.get_course_sections(selected_course.course, term=self._term)
            except Exception:
                raise AssertionError(f"{selected_course.course} is not a valid course")

            if selected_course.section:
                course_sections = list(
                    filter(lambda c: c.sequenceNumber == selected_course.section, course_sections)
                )
                assert course_sections, f"{selected_course.course} has no section numbered {selected_course.section}"
            else:
                course_sections = list(filter(section_filter, course_sections))
                assert course_sections, f"{selected_course.course} has no available sections"

            # Append list of sections for each course so that we can get the cartesian product
            all_sections.append(course_sections)

        # Get the cartesian product of all sections
        cartesian_product = list(product(*all_sections))
        return list(filter(overlap_filter, cartesian_product))


# pprint(session.get_course_sections)

builder = CourseBuilder(session)
builder.select_term(202503)

builder.update(
    [
        CourseSelect(course="MATH2043"),
        CourseSelect(course="MATH2101"),
        CourseSelect(course="CIS1068", section="001"),
        CourseSelect(course="IH0851", section="707"),

        CourseIgnore(course="IH0851", instructional_method="CLAS"),
        CourseIgnore(course="IH0851", waitlist=True),
        # CourseIgnore(course="IH0851", teacher="Patricia Moore-Martinez"),
    ]
)


pprint(builder.get_combinations())

